In [1]:
import pandas as pd
import geopandas as gpd
from h3_utils import *
import h3
import h3pandas

import ee
ee.Initialize()

import geemap

In [2]:
nga = gpd.read_file(r'..\data\Nigeria_-_State_Boundaries.zip')

In [3]:
ss = ['Abia', 'Anambra', 'Ebonyi', 'Enugu', 'Imo']

In [4]:
south_states = nga[nga['statename'].isin(ss)]

In [6]:
africapolis = gpd.read_file(r'../data/AFRICAPOLIS2020.zip')

In [8]:
south_states_gee = geemap.gdf_to_ee(south_states)

In [11]:
hex8 = hex_generator(south_states,8)

Starting...
Done
Generating hex at level 8
Done


In [21]:
glc = ee.ImageCollection('projects/glad/GLCmap2019').filterBounds(south_states_gee).mosaic()

In [22]:
hexagons_gee_list = []
if hex8.shape[0]>1000:
    chunks = int(np.ceil(hex8.shape[0]/1000))

    for i in range(1,chunks+1):
        if i!=chunks:
            temp = hex8.iloc[(i-1)*1000:i*1000]
            hexagons_gee_list.append(geemap.gdf_to_ee(temp))
        
        if i==chunks:
            temp = hex8.iloc[(i-1)*1000:]
            hexagons_gee_list.append(geemap.gdf_to_ee(temp))

else:
    hexagons_gee_list.append(geemap.gdf_to_ee(hex8))



In [23]:
len(hexagons_gee_list)

47

In [25]:
print('Start GEE')

for idx, hex_gee in enumerate(hexagons_gee_list):
    out_path = '../data/glc/hex8'
    out_dir = os.path.join(out_path)
    out_file_path = os.path.join(out_dir, f'hex8_lc_{idx}.csv')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)



    geemap.zonal_statistics(glc, hex_gee, out_file_path, statistics_type='MODE', scale=30)

Start GEE
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_0.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_1.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_2.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_3.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_4.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_5.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHub\BluePaleDot\data\glc\hex8\hex8_lc_6.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to d:\GitHu

In [27]:
hex8_glc = pd.concat([pd.read_csv(os.path.join(out_path, i)) for i in os.listdir(out_path)])

In [38]:
hex8_glc

,mode,system:index,hex8
0,252,0,8858d68f6dfffff
1,58,1,8858896365fffff
2,59,2,8882da4cb7fffff
3,50,3,8882da2ee7fffff
4,239,4,8882da7a85fffff
...,...,...,...
995,169,995,88588bb057fffff
996,60,996,88588b6307fffff
997,252,997,8858896e05fffff
998,251,998,88588ba805fffff


In [40]:
hex8 = gpd.read_file('../data/nga_ss_hex8.gpkg')

In [42]:
hex8.merge(hex8_glc[['hex8','mode']].rename(columns={'mode':'land cover'})).to_file('../data/nga_ss_hex8.gpkg')

In [29]:
hex8_glc['mode'] = hex8_glc['mode'].astype(int)

In [37]:
hex8_glc[(hex8_glc['mode']>=240)&(hex8_glc['mode']<=249)]['mode'].value_counts()

mode
240    4965
241     141
249      91
242      68
243       2
Name: count, dtype: int64

In [44]:
hex8.merge(hex8_glc[['hex8','mode']].rename(columns={'mode':'land cover'})).to_parquet('../data/nga_ss_hex8.parquet')